Petit test markdown

Imports

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

#import de Bokeh pour les cartes
from bokeh.io import output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import output_notebook, show, output_file
from bokeh.io import export_png
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, BoxZoomTool, ResetTool, SaveTool
from bokeh.palettes import brewer
output_notebook()
import json

#import pour l'API
import requests

from statistics import mean

import time

Loading BokehJS ...

In [ ]:
url="https://drive.google.com/file/d/1GGOLMc_Ow9yZC9sICegPegDggQuHOD3t/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
base = pd.read_csv(url)

Premier avant-goût de la base

In [ ]:
pd.set_option('display.max_columns', None)
base.sample(5)

Ici, nécessaire de convertir toutes les colonnes à un format manipulable : dictionnaires

In [ ]:
def convert_to_dict(value):
    if pd.isna(value):
        return value

    pairs = value.split('||')
    result_dict = {}
    for pair in pairs:
        #Some are corrupted : 1: instead of ::
        if '::' in pair:
            key, val = pair.split('::', 1)
            result_dict[int(key)] = val
        else:
            key, val = pair.split(':', 1)
            result_dict[int(key)] = val
    return result_dict

updated_base = base.copy()
list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict)
updated_base.head()



Première représentation graphique des incidents de la base

In [ ]:
us_states = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_state_20m.zip')

incident_counts = updated_base.groupby('state')['incident_id'].count().reset_index()
us_states = us_states.merge(incident_counts, left_on='NAME', right_on='state', how='left')

# Plot the map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
us_states.plot(column='incident_id', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend=False)

# Add a title and show the plot
plt.title('Gun Violence Incidents by State')
ax.set_xlim([-140, -60])  # Adjust these values based on your preference
ax.set_ylim([20, 50]) 


plt.show()

Carte interactive

In [ ]:
drop_state = ["AS", "AK", "HI", "GU", "VI", "MP", "PR"] #to keep the continental USA
us_states = us_states[~us_states["STUSPS"].isin(drop_state)]

In [ ]:
us_states_geojson = us_states[["state", "geometry", "incident_id"]].rename(columns= {"incident_id" : "nb_incidents"})
us_states_geojson = GeoJSONDataSource(geojson = us_states_geojson.to_json())

In [ ]:
color_mapper = LinearColorMapper(palette = brewer['RdBu'][10], low = 400, high = 20000)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 530, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal')
save = SaveTool()
hover = HoverTool(tooltips = [ ("State","@state"),("Nb d'incidents", "@nb_incidents")])
p = figure(title="Nombre d'incidents par Etats", tools=[hover, save], width=600, height=350)
p.patches("xs","ys",source=us_states_geojson, fill_color = {"field" :"nb_incidents", "transform" : color_mapper})
p.add_layout(color_bar, 'above')
p.axis.visible = False
show(p)
#export_png(p, filename="plot.png", width = 600, height = 350)

Fred Api to get data series about the counties

Création d'un dictionnaire contenant tous les états et des sous-dictionnaires pour chaque comtés

In [2]:
api_key = "180de2e6a1d1e953d270ebf38341cd44"
param = {"api_key" : api_key, "file_type" : "json", "category_id" : "27281"}
url = "https://api.stlouisfed.org/fred/category/children?"
response = requests.get(url, params=param)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    # Process the data as needed
else:
    print(f"Error: {response.status_code}, {response.text}")

liste = data["categories"] #on rentre dans le premier dictionnaire
dico_id_states = dict()

for dic in liste : #on parcourt la liste en stockant dans un dictionnaire le nom de l'état et le numéro de la catégorie associée
    dico_id_states[dic["name"]] = dic["id"]

In [3]:
state_relous = ["Alaska", "Louisiana","Virgin Islands"] #on retire les états qui n'ont pas de comties
for state in state_relous :
    del dico_id_states[state]

In [4]:
for state in dico_id_states.keys() : #on parcourt les états
    param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    dic = data["categories"][0] #on rentre dans le dictionnaire (toujours le premier)

    category_id = dic["id"]
    dico_id_states[state] = category_id


In [5]:
dic_states_counties = dict()

for state in dico_id_states.keys() : #on parcourt les états
    param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    liste = data["categories"]

    dic_states_counties[state] = dict() #on prépare le dictionnaire
    

    for dic in liste : #on parcourt les dictionnaires (1 par county)
        county_name = dic["name"]
        dic_states_counties[state][county_name] = {"id" : dic["id"]} #on crée une entrée dans le dic pour le county

In [ ]:
# dic_states_counties = dict()

# for state in dico_id_states.keys() : #on parcourt les états
#     param["category_id"] = dico_id_states[state] #on ajuste les paramètres de la request pour demander la bonne catégorie
#     response = requests.get(url, params = param)
#     data = response.json()
#     liste = data["categories"]

#     dic_states_counties[state] = dict() #on prépare le dictionnaire
    

#     for dic in liste : #on parcourt les dictionnaires (1 par county)
#         county_name = dic["name"]
#         dic_states_counties[state][county_name] = dict() #on crée une entrée dans le dic pour le county

Récupération du taux de chômage moyen entre 2013 et 2017 (inclus)

In [ ]:
# dic_states_counties["Illinois"]["DeKalb County, IL"] = dic_states_counties["Illinois"]["De Kalb County, IL"]
# del dic_states_counties["Illinois"]["De Kalb County, IL"]

In [ ]:
#méthode en cherchant la série
state = "Iowa"

dic_state = dic_states_counties[state] #on se place dans le dictionnaire de l'état

for county in dic_states_counties[state].keys(): #on parcourt les counties
    dic_county = dic_state[county] #on se place dans le dictionnaire du county

    url = "https://api.stlouisfed.org/fred/series/search?" #on va chercher les séries correspondant au chomage annuel
    param = {"api_key" : api_key, "file_type" : "json"}
    param["search_text"] = "annual+unemployment+rate+" + county.replace(",", "").replace(" ", "+") #on écrit les termes de la recherche, prblème avec les filtres
    response = requests.get(url, params= param)
    data = response.json()
    

    #print(data)
    #print(county)
    #print(response.url)

    unemployment_rate_serie_id = data["seriess"][0]["id"]
    dic_county["unemployment_rate_id"] = unemployment_rate_serie_id

    time.sleep(1)


    #récupération de la série en commentaire à cause de la limite de requests par minute
    # url = "https://api.stlouisfed.org/fred/series/observations?" #on va chercher les séries correspondant au chomage annuel
    # param = {"api_key" : api_key, "file_type" : "json", "series_id" : unemployment_rate_serie_id, "observation_start" : "2013-01-01", "observation_end" : "2017-01-01"} 
    
    # response = requests.get(url, params= param)
    # data = response.json()
    
    # print(response.url)

    # liste_observations = [float(obs["value"]) for obs in data["observations"]]
    # unemployment_rate_mean = mean(liste_observations)

    # dic_county["unemployment_rate"] = unemployment_rate_mean 

Autre méthode : on cherche la série avec "unemployment rate" dans le titre et en fréquence mensuelle

In [6]:
counties_df = pd.DataFrame()

for state in dic_states_counties.keys():
    dic_state = dic_states_counties[state] #on se place dans le dictionnaire de l'état

    for county in dic_states_counties[state].keys(): #on parcourt les counties
        dic_county = dic_state[county] #on se place dans le dictionnaire du county

        url = "https://api.stlouisfed.org/fred/category/series?"
        param = {"api_key" : api_key, "file_type" : "json", "category_id" : dic_county["id"]} #on va accéder à la page contenant les séries du county

        response = requests.get(url, params= param)
        data = response.json()     
        
        #maintenant on cherche la série dont le titre contient "unemployment rate" et dont la fréquence est annuelle
        liste_series = data["seriess"]
        
        for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
            if ("Unemployment Rate" in serie["title"]) and ("Monthly" in serie["frequency"]):
                dic_county["unemployment_rate_id"] = serie["id"]  #on récupère l'id de la série et on le stock
        time.sleep(0.5)

    for county in dic_states_counties[state].keys(): #on parcourt les counties
        dic_county = dic_state[county] #on se place dans le dictionnaire du county

        #récupération de la série
        if "unemployment_rate_id" in dic_county.keys() :

            url = "https://api.stlouisfed.org/fred/series/observations?" #on va chercher les séries correspondant au chomage mensuel
            param = {"api_key" : api_key, "file_type" : "json", "series_id" : dic_county["unemployment_rate_id"], "observation_start" : "2013-01-01", "observation_end" : "2018-03-01"} 
        
            response = requests.get(url, params= param)
            data = response.json()

            liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
            
            unemployment_rate_mean = mean(liste_observations)

            dic_county["state"] = state
            dic_county["unemployment_rate"] = unemployment_rate_mean

        time.sleep(0.5)

    df_temp = pd.DataFrame.from_dict(dic_states_counties[state], orient = "index")
    counties_df = pd.concat([counties_df, df_temp])

KeyboardInterrupt: 

In [ ]:
counties_df.to_csv(path_or_buf="counties_data2.csv")

In [ ]:
# méthode en cherchant dans la catégorie du county :
# state = "Kansas"

# dic_state = dic_states_counties[state] #on se place dans le dictionnaire de l'état

# for county in dic_states_counties[state].keys(): #on parcourt les counties
#     dic_county = dic_state[county] #on se place dans le dictionnaire du county

#     url = "https://api.stlouisfed.org/fred/category/series?"
#     param = {"api_key" : api_key, "file_type" : "json", "category_id" : dic_county["id"]} #on va accéder à la page contenant les séries du county

#     response = requests.get(url, params= param)
#     data = response.json()
    
    

#     #maintenant on cherche la série dont le titre contient "unemployment rate" et donc la fréquence est annuelle
#     liste_series = data["seriess"]
    
#     for serie in liste_series: #on parcourt les séries,en cherchant celle qui parle du taux de chomage annuel
#         if ("Unemployment Rate" in serie["title"]) and ("Annual" in serie["frequency"]):
#             dic_county["unemployment_rate_id"] = serie["id"]  #on récupère l'id de la série et on le stock
#     time.sleep(0.5)
    

In [ ]:
# dic_state = dic_states_counties[state] #on se place dans le dictionnaire de l'état

# for county in dic_states_counties[state].keys(): #on parcourt les counties
#     dic_county = dic_state[county] #on se place dans le dictionnaire du county

#     #récupération de la série
#     if "unemployment_rate_id" in dic_county.keys() :

#         url = "https://api.stlouisfed.org/fred/series/observations?" #on va chercher les séries correspondant au chomage annuel
#         param = {"api_key" : api_key, "file_type" : "json", "series_id" : dic_county["unemployment_rate_id"], "observation_start" : "2013-01-01", "observation_end" : "2017-01-01"} 
    
#         response = requests.get(url, params= param)
#         data = response.json()

#         liste_observations = [float(obs["value"]) for obs in data["observations"]] #on récupère toutes les observations pour en faire la moyenne sur la période
        
#         unemployment_rate_mean = mean(liste_observations)

#         dic_county["state"] = state
#         dic_county["unemployment_rate"] = unemployment_rate_mean

#     time.sleep(0.5)


In [ ]:
counties_df

In [ ]:
# df1 = pd.DataFrame.from_dict(dic_states_counties[state], orient = "index")
# df1["state"] = state
# df = pd.concat([df, df1])
# df.to_csv(path_or_buf="lol2.csv")
# df.tail(5)

### Problème méthode recherche (en gros pas d'assurance d'avoir le bon comté)
Attention : erreur pour le district de Columbia, la vraie série sur  le chomage est LAUCN110010000000003A

Pas de hawaii car marche bof

Ajustement manuel pour le "De Kalb County, IL" en Illinois en remplacant par "DeKalb County, IL"

Washington County, IN faux : vrai id est : LAUCN181750000000003A et non LAUCN110010000000003A

Wayne County, IN faux aussi shit

### Problème méthode recherche : pas de données pour certains comtés
Executin totale : 76 min